# Mini Project 2 - DL Skills - Modeling

## Import libraries and load data

In [ ]:
import keras
import tensorflow as tf
from utils import (
    get_category,
    get_prepared_data
)

In [ ]:
(x_train, x_test), (y_train, y_test) = get_prepared_data()

In [ ]:
batch_size = 16

# https://www.tensorflow.org/tutorials/load_data/numpy
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(50000, reshuffle_each_iteration=True)
train_dataset = train_dataset.batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat(4)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(10000, reshuffle_each_iteration=True)
test_dataset = test_dataset.batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.repeat(4)

## Model Network

### Model: VGG

In [ ]:
from model_utils import get_vgg

model = get_vgg()

model.summary()

In [ ]:
from keras.optimizers import adam_v2

opt = adam_v2.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True
)

model.compile(
    optimizer=opt,
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import (ModelCheckpoint, EarlyStopping)
checkpoint = ModelCheckpoint(
    "updated_vgg.h5",
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(
    x=train_dataset,
    steps_per_epoch=100,
    validation_data=test_dataset,
    validation_steps=10,
    epochs=100,
    callbacks=[checkpoint, early]
)

### Plot Learning Curves (Loss and Accuracy)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(hist.history["loss"])
ax1.plot(hist.history["val_loss"])
ax1.set_title("model loss")
ax1.set_ylabel("loss")
ax1.set_xlabel("epoch")
ax1.legend(["train", "test"], loc="upper left")

ax2.plot(hist.history["accuracy"])
ax2.plot(hist.history["val_accuracy"])
ax2.set_title("model accuracy")
ax2.set_ylabel("accuracy")
ax2.set_xlabel("epoch")
ax2.legend(["train", "test"], loc="upper left")

fig.tight_layout()
plt.show()

### Predictions

In [ ]:
# Array predictions
y_hat = model.predict(x_test)

# Single prediction
# y_hat_2000 = model.predict(x_test[2000])

In [ ]:
import numpy as np
from PIL import Image

# Recheck accuracy calculation on test set
count = 0
for i in range(len(y_test)):
    if np.argmax(y_test[i]) == np.argmax(y_hat[i]):
        count += 1
print(f'Matches: {count} out of {len(y_test)}')

# Single prediction
# print(get_category(y_test[2000], verbose=True))
# print(get_category(y_hat_2000, verbose=True))

# Array predictions
test_idx = 30
print(get_category(y_test[test_idx], verbose=True))
print(get_category(y_hat[test_idx], verbose=True))
Image.fromarray((x_test[test_idx] * 255.0).reshape(32, 32)).convert('L').resize((128, 128))